In [1]:
import numpy as np

In [141]:
def gauss_method_column(a : np.ndarray[np.ndarray[float]], b :np.ndarray[float], show_annotation : bool = False) -> np.ndarray[float]:
    if a.shape[0] != b.shape[0]:
        raise ValueError("The dimensions of the two arrays must match")
    
    det_a = np.linalg.det(a)
    if abs(det_a) < 1e-12:
        raise ValueError("Determinant is zero — system has no unique solution")
    
    n = len(b)
    a = a.astype(float)
    b = b.astype(float)
    if show_annotation:
        print("System: ", a, b, "\n")
    
    order = np.array([-1] * n)
    
    for i in range(n):
        max_row = -1
        max_val = 0
        
        for j in range(n):
            if j in order[:i]:
                continue
            if abs(a[j][i]) > abs(max_val):
                max_val = a[j][i]
                max_row = j
        
        if max_row == -1 or abs(max_val) < 1e-12:
            raise ValueError("Zero column detected — system has no unique solution")
        
        o = max_row
        order[i] = o
        lead_el = a[o][i]
        
        for j in range(n):    
            if j in order[:i+1]:
                continue
            if j == o:
                continue
            factor = a[j][i] / lead_el
            a[j] = a[j] - factor * a[o]
            b[j] = b[j] - factor * b[o]
            a[j][i] = 0
        
        a[o] = a[o] / lead_el
        b[o] = b[o] / lead_el
        a[o][i] = 1
        
        if show_annotation:
            print(f"Step {i + 1}: \n Row chosen: ", o + 1, "\n", a, b, "\n")
    
    ans = np.zeros(n)
    for i in range(n - 1, -1, -1):
        row_num = order[i]
        xi = b[row_num]
        for j in range(i + 1, n):
            xi -= a[row_num][j] * ans[j]
        ans[i] = xi
    return ans

In [142]:
a = np.array([[2, 3, 2], [2, 3, 4], [3, 4, 5]])
b = np.array([7, 9, 12])
gauss_method_column(a, b, show_annotation=True)

System:  [[2. 3. 2.]
 [2. 3. 4.]
 [3. 4. 5.]] [ 7.  9. 12.] 

Step 1: 
 Row chosen:  3 
 [[ 0.          0.33333333 -1.33333333]
 [ 0.          0.33333333  0.66666667]
 [ 1.          1.33333333  1.66666667]] [-1.  1.  4.] 

Step 2: 
 Row chosen:  1 
 [[ 0.          1.         -4.        ]
 [ 0.          0.          2.        ]
 [ 1.          1.33333333  1.66666667]] [-3.  2.  4.] 

Step 3: 
 Row chosen:  2 
 [[ 0.          1.         -4.        ]
 [ 0.          0.          1.        ]
 [ 1.          1.33333333  1.66666667]] [-3.  1.  4.] 



array([1., 1., 1.])

In [147]:
systems = [
    (np.array([[2, 3], [1, -1]], dtype=float), np.array([8, 0], dtype=float)),
    
    (np.array([[2, 3, 2],
               [2, 3, 4],
               [3, 4, 5]], dtype=float), np.array([7, 9, 12], dtype=float)),
    
    (np.array([[4, 2, 0, 1],
               [3, 1, -1, 2],
               [2, -1, 3, 1],
               [1, 0, 2, 1]], dtype=float), np.array([10, 8, 7, 3], dtype=float)),
    
    (np.array([[2, 1, 3, 2, 4],
               [1, 2, 0, 1, 3],
               [3, 0, 2, 1, 1],
               [2, 1, 1, 3, 2],
               [1, 3, 2, 0, 1]], dtype=float), np.array([10, 6, 7, 9, 5], dtype=float)),
    
    (np.array([[1, 2, 1, 3, 2, 1],
               [2, 1, 3, 1, 2, 1],
               [3, 2, 1, 2, 1, 3],
               [1, 3, 2, 1, 3, 2],
               [2, 1, 2, 3, 1, 2],
               [1, 2, 3, 1, 2, 1]], dtype=float), np.array([10, 11, 12, 13, 14, 15], dtype=float)),
]


np.random.seed(42)
systems.append((np.random.rand(10, 10) * 10, np.random.rand(10) * 10))
systems.append((np.random.rand(20, 20) * 10, np.random.rand(20) * 10))

for idx, (A, b) in enumerate(systems, start=1):
    print(f"\n{A.shape[0]}x{A.shape[1]}")
    try:
        result = gauss_method_column(A.copy(), b.copy())
        expected = np.linalg.solve(A, b)
        print("Максимальная абсолютная погрешность:", np.max(np.abs(result - expected)))
        if np.allclose(result, expected):
            print("OK")
        else:
            print("Ответы отличаются")
    except ValueError as e:
        print(e)



2x2
Максимальная абсолютная погрешность: 0.0
OK

3x3
Максимальная абсолютная погрешность: 2.220446049250313e-16
OK

4x4
Максимальная абсолютная погрешность: 0.0
OK

5x5
Максимальная абсолютная погрешность: 4.440892098500626e-16
OK

6x6
Максимальная абсолютная погрешность: 3.552713678800501e-15
OK

10x10
Максимальная абсолютная погрешность: 1.1102230246251565e-15
OK

20x20
Максимальная абсолютная погрешность: 3.6415315207705135e-14
OK


In [154]:
import time
def test_large_system(n):
    print(f"\n{n}x{n}")
    np.random.seed(42)
    A = np.random.rand(n, n) * 10
    b = np.random.rand(n) * 10

    start = time.time()
    result = gauss_method_column(A.copy(), b.copy())
    elapsed_gauss = time.time() - start
    
    start = time.time()
    expected = np.linalg.solve(A, b)
    elapsed_numpy = time.time() - start

    max_error = np.max(np.abs(result - expected))
    print(f"Максимальная абсолютная погрешность: {max_error:.2e}")
    print(f"Время Гаусс = {elapsed_gauss:.4f}s, NumPy = {elapsed_numpy:.4f}s")
    if np.allclose(result, expected):
        print("ОК")
    else:
        print("Ответы отличаются")

test_large_system(50)
test_large_system(100)
test_large_system(300)
test_large_system(500)
test_large_system(800)
test_large_system(1000)


50x50
Максимальная абсолютная погрешность: 4.55e-15
Время Гаусс = 0.0297s, NumPy = 0.0000s
ОК

100x100
Максимальная абсолютная погрешность: 9.96e-15
Время Гаусс = 0.0864s, NumPy = 0.0000s
ОК

300x300
Максимальная абсолютная погрешность: 3.70e-13
Время Гаусс = 1.3821s, NumPy = 0.0000s
ОК

500x500
Максимальная абсолютная погрешность: 2.98e-13
Время Гаусс = 3.0793s, NumPy = 0.0168s
ОК

800x800
Максимальная абсолютная погрешность: 6.45e-13
Время Гаусс = 6.0802s, NumPy = 0.0179s
ОК

1000x1000
Максимальная абсолютная погрешность: 2.22e-12
Время Гаусс = 9.8460s, NumPy = 0.0359s
ОК
